In [1]:
import sys
print('Python version: ', sys.version)

Python version:  3.6.10 |Anaconda, Inc.| (default, Mar 25 2020, 18:53:43) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import folium

# 1. Position Data

In [3]:
# data from https://github.com/chriswhong/nycturnstiles/blob/master/geocoded.csv
geo_df = pd.read_csv('../data/turnstile/geocoded_ca_unit.csv').dropna()
geo_df

,remote unit,control area,station,lines,division,latitude,longitude
0,R470,X002,ELTINGVILLE PK,Z,SRT,40.544600,-74.164581
1,R544,PTH02,HARRISON,1,PTH,40.738879,-74.155533
2,R165,S102,TOMPKINSVILLE,1,SRT,40.636948,-74.074824
3,R070,S101,ST. GEORGE,1,SRT,40.643738,-74.073622
4,R070,S101A,ST. GEORGE,1,SRT,40.643738,-74.073622
...,...,...,...,...,...,...,...
789,R001,R101,SOUTH FERRY,1RW,IRT,40.702068,-74.013664
790,R305,R107D,CORTLANDT ST,1,IRT,40.710454,-74.011324
791,R305,R108A,CORTLANDT ST,1,IRT,40.710454,-74.011324
792,R072,R550,34 ST-HUDSON YD,7,IRT,40.755839,-74.001961


In [4]:
# Build map 
station_loc_map = folium.Map(location=[40.738, -73.98],
    zoom_start=11, tiles='cartodbpositron')

# Plot coordinates using comprehension list
for index, row in geo_df.iterrows():
    folium.CircleMarker(location=[row['latitude'], row['longitude']],
    color='#0080bb', fill_color='#0080bb', radius=1).add_to(station_loc_map) 

# Display map in Jupyter
station_loc_map

# 2. Trip Data from trunstile_cleaning_functoins

In [5]:
%%time
cleaned_files = ['../data/turnstile/turnstile_diffs_2019.csv.zip',
                 '../data/turnstile/turnstile_diffs_2020.csv.zip']
count_df = pd.concat(pd.read_csv(f, low_memory=False) for f in cleaned_files)
count_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8123410 entries, 0 to 3776137
Data columns (total 9 columns):
 #   Column    Dtype  
---  ------    -----  
 0   C/A       object 
 1   UNIT      object 
 2   SCP       object 
 3   STATION   object 
 4   LINENAME  object 
 5   DIVISION  object 
 6   datetime  object 
 7   entries   float64
 8   exits     float64
dtypes: float64(2), object(7)
memory usage: 619.8+ MB
CPU times: user 7.92 s, sys: 1.91 s, total: 9.83 s
Wall time: 9.88 s


In [6]:
station_df = count_df[['C/A', 'UNIT', 'STATION', 'LINENAME', 'DIVISION']].drop_duplicates()
print(len(station_df), 'entries in turnstile data')
station_df.head()

750 entries in turnstile data


,C/A,UNIT,STATION,LINENAME,DIVISION
0,A002,R051,59 ST,NQR456W,BMT
10697,A006,R079,5 AV/59 ST,NQRW,BMT
18016,A007,R079,5 AV/59 ST,NQRW,BMT
23433,A010,R080,57 ST-7 AV,NQRW,BMT
30801,A011,R080,57 ST-7 AV,NQRW,BMT


# 3. Join

In [7]:
# join by C/A and UNIT
geostation_df = pd.merge(station_df, geo_df, how='outer',
    left_on=['C/A', 'UNIT'], right_on=['control area', 'remote unit'], suffixes=['_l', '_r'])
geostation_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 786 entries, 0 to 785
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   C/A           750 non-null    object 
 1   UNIT          750 non-null    object 
 2   STATION       750 non-null    object 
 3   LINENAME      750 non-null    object 
 4   DIVISION      750 non-null    object 
 5   remote unit   776 non-null    object 
 6   control area  776 non-null    object 
 7   station       776 non-null    object 
 8   lines         776 non-null    object 
 9   division      776 non-null    object 
 10  latitude      776 non-null    float64
 11  longitude     776 non-null    float64
dtypes: float64(2), object(10)
memory usage: 79.8+ KB


## Station locations matched

In [8]:
geostation_nona_df = geostation_df.dropna()
print(len(geostation_nona_df), 'entries have location matched')

740 entries have location matched


In [9]:
# Build map 
station_loc_map = folium.Map(location=[40.738, -73.98],
    zoom_start=11, tiles='cartodbpositron', width=640, height=480)

# Plot coordinates using comprehension list
for index, row in geostation_nona_df.iterrows():
    folium.CircleMarker(location=[row['latitude'], row['longitude']],
    color='#0080bb', fill_color='#0080bb', radius=1).add_to(station_loc_map) 

# Display map in Jupyter
station_loc_map

## Station locations not found

In [10]:
geo_not_found = geostation_df[geostation_df['control area'].isnull()][['C/A', 'UNIT', 'STATION', 'LINENAME', 'DIVISION']]
geo_not_found.sort_values('STATION')

,C/A,UNIT,STATION,LINENAME,DIVISION
446,PTH10,R547,9TH STREET,1,PTH
444,PTH07,R550,CITY / BUS,1,PTH
450,PTH16,R550,LACKAWANNA,1,PTH
452,PTH18,R549,NEWARK BM BW,1,PTH
453,PTH19,R549,NEWARK C,1,PTH
454,PTH20,R549,NEWARK HM HE,1,PTH
438,PTH01,R549,NEWARK HW BMEBE,1,PTH
456,PTH22,R540,PATH NEW WTC,1,PTH
443,PTH06,R546,PAVONIA/NEWPORT,1,PTH
449,PTH13,R541,THIRTY ST,1,PTH


## Complementing location data (manually added to geolocation data)

In [11]:
station_name = '34 ST-HUDSON YD'
geo_not_found[geo_not_found['STATION'] == station_name]

,C/A,UNIT,STATION,LINENAME,DIVISION


In [12]:
geostation_nona_df[geostation_nona_df['STATION'] == station_name]

,C/A,UNIT,STATION,LINENAME,DIVISION,remote unit,control area,station,lines,division,latitude,longitude
699,R550,R072,34 ST-HUDSON YD,7,IRT,R072,R550,34 ST-HUDSON YD,7,IRT,40.755839,-74.001961
700,R551,R072,34 ST-HUDSON YD,7,IRT,R072,R551,34 ST-HUDSON YD,7,IRT,40.755839,-74.001961


In [13]:
station_df[station_df['STATION'] == station_name]

,C/A,UNIT,STATION,LINENAME,DIVISION
4063095,R550,R072,34 ST-HUDSON YD,7,IRT
4079730,R551,R072,34 ST-HUDSON YD,7,IRT


# 4. Station definition

In [14]:
# TODO: check joined result
# geostation_nona_df.loc[geostation_nona_df.DIVISION != geostation_nona_df.division]
# right now, just drop repeated columns
geostation = geostation_nona_df.loc[:,['C/A', 'UNIT', 'STATION', 'LINENAME', 'DIVISION', 'latitude', 'longitude']]
# sort linenames by alphabet
geostation['lines'] = geostation.LINENAME.apply(lambda name: ''.join(sorted(name)))
geostation.nunique()

C/A          740
UNIT         466
STATION      369
LINENAME     114
DIVISION       6
latitude     451
longitude    450
lines         95
dtype: int64

In [15]:
# group by station name and lines
# station my contain multiple C/A UNIT
geostation[['C/A', 'UNIT', 'STATION', 'LINENAME']].groupby(
    ['STATION', 'LINENAME'], as_index=False).size().reset_index(name='ca_unit_cnt').sort_values(
    'ca_unit_cnt', ascending=False).head()

,STATION,LINENAME,ca_unit_cnt
94,34 ST-PENN STA,ACE,7
311,GRD CNTRL-42 ST,4567S,7
301,FULTON ST,2345ACJZ,7
466,WTC-CORTLANDT,1,6
90,34 ST-HERALD SQ,BDFMNQRW,5


In [16]:
# station may have multiple locations 
geostation[['STATION', 'lines', 'latitude', 'longitude']].sort_values(['STATION', 'lines']).head()

,STATION,lines,latitude,longitude
124,1 AV,L,40.730901,-73.981719
125,1 AV,L,40.730901,-73.981719
749,1 AV,L,40.730901,-73.981719
518,103 ST,1,40.799354,-73.968329
588,103 ST,6,40.790582,-73.947473


In [17]:
unique = geostation[['STATION', 'lines', 'latitude', 'longitude']].groupby(
    ['STATION', 'lines']).nunique(
    ['latitude', 'longitude'])[['latitude', 'longitude']].reset_index().rename(
    columns={"latitude": "lat_cnt", "longitude": "lon_cnt"})
unique = unique.loc[(unique['lat_cnt'] != 1) | (unique['lon_cnt'] != 1)] # any unexpected value?
unique

,STATION,lines,lat_cnt,lon_cnt
24,14 ST,123FLM,2,2
70,23 ST,FM,2,2
76,25 ST,R,2,2
226,CANAL ST,6JNQRWZ,2,2
315,HOYT ST,23,2,2


In [18]:
duplicated = pd.merge(geostation[['STATION', 'lines', 'DIVISION', 'latitude', 'longitude']], unique)
duplicated

,STATION,lines,DIVISION,latitude,longitude,lat_cnt,lon_cnt
0,CANAL ST,6JNQRWZ,BMT,40.718697,-74.000977,2,2
1,CANAL ST,6JNQRWZ,BMT,40.718697,-74.000977,2,2
2,CANAL ST,6JNQRWZ,BMT,40.718233,-74.000323,2,2
3,25 ST,R,BMT,40.660481,-73.998059,2,2
4,25 ST,R,BMT,40.660430,-73.997944,2,2
5,23 ST,FM,IND,40.742981,-73.992727,2,2
6,23 ST,FM,IND,40.742868,-73.992770,2,2
7,14 ST,123FLM,IND,40.737348,-73.996900,2,2
8,14 ST,123FLM,IND,40.737348,-73.996900,2,2
9,14 ST,123FLM,IND,40.737348,-73.996900,2,2


In [19]:
# Build map 
station_loc_map = folium.Map(location=[40.738, -73.98],
    zoom_start=11, tiles='cartodbpositron')

# Plot coordinates using comprehension list
for index, row in duplicated.iterrows():
    folium.CircleMarker(location=[row['latitude'], row['longitude']],
    color='#0080bb', fill_color='#0080bb', radius=1).add_to(station_loc_map) 

# Display map in Jupyter
station_loc_map

In [20]:
# TODO: fix duplication caused by different division and lines
# rigth now, computer the average location
station_line_mean_loc = geostation[['STATION', 'lines', 'latitude', 'longitude']].groupby(
    ['STATION', 'lines'], as_index=False).agg({'latitude':'mean', 'longitude':'mean'})
averaged = pd.merge(station_line_mean_loc, unique)
averaged

,STATION,lines,latitude,longitude,lat_cnt,lon_cnt
0,14 ST,123FLM,40.737568,-73.997394,2,2
1,23 ST,FM,40.742925,-73.992749,2,2
2,25 ST,R,40.660455,-73.998001,2,2
3,CANAL ST,6JNQRWZ,40.718542,-74.000759,2,2
4,HOYT ST,23,40.690546,-73.985066,2,2


In [21]:
# Build map 
station_loc_map = folium.Map(location=[40.738, -73.98],
    zoom_start=11, tiles='cartodbpositron')

# Plot coordinates using comprehension list
for index, row in averaged.iterrows():
    folium.CircleMarker(location=[row['latitude'], row['longitude']],
    color='#0080bb', fill_color='#0080bb', radius=1).add_to(station_loc_map) 

# Display map in Jupyter
station_loc_map

# 5. Zip code, Neighborhood, and Borough

## Get zipcode from latitude/longitude

In [22]:
# # using geopy
# # this is extremely slow!! And might not find the answer
# # https://gis.stackexchange.com/questions/352961/convert-lat-lon-to-zip-postal-code-using-python
# import geopy
# def get_zipcode(df, geolocator, lat_field, lon_field):
#     location = geolocator.reverse((df[lat_field], df[lon_field]))
#     if 'postcode' in location.raw['address']:
#         return location.raw['address']['postcode']
#     print('postcode not found:\n', location.raw['address'] )
#     return float('nan')
# geolocator = geopy.Nominatim(user_agent='my-application')
# zipcodes = station_line_mean_loc.apply(get_zipcode, axis=1, geolocator=geolocator,
#     lat_field='latitude', lon_field='longitude')
# zipcodes

In [23]:
# using uszipcode
from uszipcode import SearchEngine
search = SearchEngine(simple_zipcode=True)
from uszipcode import Zipcode
def get_zipcode(lat, lon):
    result = search.by_coordinates(lat = lat, lng = lon, returns = 1)
    return result[0]

zipcodes = station_line_mean_loc.apply(
    lambda x: get_zipcode(x.latitude,x.longitude), axis=1)
zipcodes

0      SimpleZipcode(zipcode='10009', zipcode_type='S...
1      SimpleZipcode(zipcode='10025', zipcode_type='S...
2      SimpleZipcode(zipcode='10029', zipcode_type='S...
3      SimpleZipcode(zipcode='10162', zipcode_type='S...
4      SimpleZipcode(zipcode='11368', zipcode_type='S...
                             ...                        
454    SimpleZipcode(zipcode='10467', zipcode_type='S...
455    SimpleZipcode(zipcode='10007', zipcode_type='S...
456    SimpleZipcode(zipcode='10271', zipcode_type='S...
457    SimpleZipcode(zipcode='11201', zipcode_type='S...
458    SimpleZipcode(zipcode='10462', zipcode_type='S...
Length: 459, dtype: object

In [24]:
regioned_loc =  station_line_mean_loc.copy(deep=True)
regioned_loc['zipcode'] = zipcodes.apply(lambda d: d.zipcode)
regioned_loc['state'] = zipcodes.apply(lambda d: d.state)
regioned_loc.head()

,STATION,lines,latitude,longitude,zipcode,state
0,1 AV,L,40.730901,-73.981719,10009,NY
1,103 ST,1,40.799354,-73.968329,10025,NY
2,103 ST,6,40.790582,-73.947473,10029,NY
3,103 ST,BC,40.796105,-73.961399,10162,NY
4,103 ST-CORONA,7,40.749858,-73.862672,11368,NY


In [25]:
# a few stations are new jersy
regioned_loc.groupby('state').size()

state
NJ      4
NY    455
dtype: int64

## Add NYC neighborhood information

In [26]:
zipcode_df = pd.read_csv('../data/nyc_zipcode_neighborhood.csv')
zipcode_df=zipcode_df.astype(str)
zipcode_df.nunique()

borough           5
neighborhood     42
zipcode         194
dtype: int64

In [27]:
allinfo_loc = pd.merge(regioned_loc,zipcode_df, how='left')
allinfo_loc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 459 entries, 0 to 458
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   STATION       459 non-null    object 
 1   lines         459 non-null    object 
 2   latitude      459 non-null    float64
 3   longitude     459 non-null    float64
 4   zipcode       459 non-null    object 
 5   state         459 non-null    object 
 6   borough       455 non-null    object 
 7   neighborhood  455 non-null    object 
dtypes: float64(2), object(6)
memory usage: 32.3+ KB


In [28]:
# after manually fixing the data
zip_na = allinfo_loc.loc[allinfo_loc.borough.isnull()]
zip_na.sort_values('zipcode')

,STATION,lines,latitude,longitude,zipcode,state,borough,neighborhood
310,HARRISON,1,40.738879,-74.155533,07102,NJ,NaN,NaN
304,GROVE STREET,1,40.719876,-74.042616,07302,NJ,NaN,NaN
330,JOURNAL SQUARE,1,40.732102,-74.063915,07306,NJ,NaN,NaN
275,EXCHANGE PLACE,1,40.716737,-74.033024,07311,NJ,NaN,NaN


In [29]:
# Build map 
station_loc_map = folium.Map(location=[40.738, -73.98],
    zoom_start=11, tiles='cartodbpositron')

# Plot coordinates using comprehension list
for index, row in zip_na.iterrows():
    folium.CircleMarker(location=[row['latitude'], row['longitude']],
    color='#0080bb', fill_color='#0080bb', radius=1).add_to(station_loc_map) 

# Display map in Jupyter
station_loc_map

In [30]:
# keep NJ stations, set to empty
allinfo_loc = allinfo_loc.fillna('')
allinfo_loc.head()

,STATION,lines,latitude,longitude,zipcode,state,borough,neighborhood
0,1 AV,L,40.730901,-73.981719,10009,NY,Manhattan,Lower East Side
1,103 ST,1,40.799354,-73.968329,10025,NY,Manhattan,Upper West Side
2,103 ST,6,40.790582,-73.947473,10029,NY,Manhattan,East Harlem
3,103 ST,BC,40.796105,-73.961399,10162,NY,Manhattan,-
4,103 ST-CORONA,7,40.749858,-73.862672,11368,NY,Queens,West Queens


In [31]:
# save to file
allinfo_loc.to_csv('../data/turnstile/geocoded_station_lines.csv')